In [1]:
import pyodbc
import pandas as pd

In [2]:
(r'DRIVER={ODBC Driver 17 for SQL Server};' +
        ('SERVER={server},{port};'   +
        'DATABASE={database};'      +
        'UID={username};'           +
        'PWD={password}').format(
                server= '65.175.68.34',
                  port= 1433,
              database= 'VsaDev',
              username= 'sa',
              password= 'H4ZXZy-vRZwL#9A')
        )

config = dict(server= '65.175.68.34',
                  port= 1433,
              database= 'VsaDev',
              username= 'sa',
              password= 'H4ZXZy-vRZwL#9A')

conn_str = ('SERVER={server},{port};'   +
            'DATABASE={database};'      +
            'UID={username};'           +
            'PWD={password}')
conn = pyodbc.connect(
    r'DRIVER={ODBC Driver 13 for SQL Server};' +
    conn_str.format(**config)
    )

In [3]:
data = ("SELECT g.GeneratedPlanID,\
        g.ParameterSetID,\
        p.MajorID,\
        m.Name as PreferedMajor, \
        p.SchoolID,\
        s.Name as PreferedSchool,\
        p.JobTypeID,\
        j.JobType,\
        QuarterPreferenceID,\
        q.Quarter as PreferedStartQuarter,\
        gp.StartingQuarter,\
        q1.Quarter as ActualStartQuarter,\
        NumberCoreCoursesPerQuarter,\
        gp3.NumberOfCorePerQuarter as ActualNumberofCore,\
        MaxNumberOfQuarters as PreferedNumberQuarters,\
        gp.NumberOfQuarters as ActualNumberQuarters,\
        CreditsPerQuarter,\
        SummerPreference as SummerPref,\
        gp2.SummerClass as ActualSummerPref,\
        p.EnrollmentTypeID,\
        e.EnrollmentDescription as EnrollmentType,\
        gp.TotalNumberOfCourses,\
        m1.MathStarting,\
        eng.EnglishStarting,\
        g.MachineScore\
        FROM VsaDev.dbo.ParameterSet as p\
        INNER JOIN VsaDev.dbo.Major as m ON p.MajorID = m.MajorID\
        INNER JOIN VsaDev.dbo.School as s ON p.SchoolID = s.SchoolID\
        INNER Join VsaDev.dbo.JobType as j on p.JobTypeID = j.JobTypeID\
        INNER JOIN VsaDev.dbo.Quarter as q on p.QuarterPreferenceID = q.QuarterID\
        INNER JOin VsaDev.dbo.EnrollmentType as e on p.EnrollmentTypeID = e.EnrollmentTypeID\
        Inner join VsaDev.dbo.GeneratedPlan as g on g.ParameterSetID = p.ParameterSetID \
        Inner JOin \
        (select s.GeneratedPlanID,\
        Count (distinct s.QuarterID) as NumberOfQuarters,\
        Count(s.CourseID)  as TotalNumberOfCourses,\
         MIN(QuarterID) as StartingQuarter\
        from VsaDev.dbo.StudyPlan as s\
        GROUP by s.GeneratedPlanID ) as gp on gp.GeneratedPlanID = g.GeneratedPlanID\
        Inner join VsaDev.dbo.Quarter as q1 on gp.StartingQuarter = q1.QuarterID\
        Full join \
        (select * from \
        (select  s.GeneratedPlanID,case when s.QuarterID = 4 then 'Yes'\
        end as SummerClass\
        from StudyPlan as s\
        GROUP by s.QuarterID,s.GeneratedPlanID)\
        as d where d.SummerClass = 'Yes') as gp2 on gp2.GeneratedPlanID = g.GeneratedPlanID\
        full JOIN\
        (select  s1.GeneratedPlanID, Count(s1.NumberOfCoursesPerQuarter)as NumberOfCorePerQuarter\
        from (\
        select  s.GeneratedPlanID,\
        s.QuarterID,\
        count(s.CourseID) as NumberOfCoursesPerQuarter\
        FROM VsaDev.dbo.StudyPlan as s\
        where s.CourseID in (select CourseID from Course as c where c.CourseNumber like 'MATH%' UNION select CourseID from Course as co1 where co1.CourseNumber like 'ENGL%')\
        GROUP by s.QuarterID,s.GeneratedPlanID)as s1\
        GROUP by s1.GeneratedPlanID) as gp3 on gp3.GeneratedPlanID = g.GeneratedPlanID\
        FULL JOIN\
        (select  s.GeneratedPlanID,\
        MIN(s.CourseID) as MathStarting\
        FROM VsaDev.dbo.StudyPlan as s\
        where s.CourseID in (select CourseID from Course as c where\
        c.CourseNumber like 'MATH%')\
        GROUP by s.GeneratedPlanID) as m1 on m1.GeneratedPlanID = g.GeneratedPlanID\
        full JOIN\
        (select  s.GeneratedPlanID,\
        MIN(s.CourseID) as EnglishStarting\
        FROM VsaDev.dbo.StudyPlan as s\
        where s.CourseID in (select CourseID from Course as c where c.CourseNumber like 'ENGL%')\
        GROUP by s.GeneratedPlanID) as eng on eng.GeneratedPlanID = g.GeneratedPlanID;")



In [4]:
vaaData = pd.read_sql(data, conn) 

In [5]:
vaaData.shape

(295, 25)

In [6]:
vaaData = vaaData.drop('ParameterSetID',1)
vaaData = vaaData.drop('MajorID',1)
vaaData = vaaData.drop('SchoolID',1)
vaaData = vaaData.drop('JobTypeID',1)
vaaData = vaaData.drop('QuarterPreferenceID',1)
vaaData = vaaData.drop('StartingQuarter',1)
vaaData = vaaData.drop('EnrollmentTypeID',1)


In [7]:
import CalculateSequenceBreak as cb

math break= 79
english break = 67


/Users/sgaruda/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


     GeneratedPlanID  MathSequenceBreak
0                  1                 71
1                  3                 68
2                  4                100
3                  5                100
4                  6                  0
5                  7                 72
6                  8                 36
7                  9                100
8                 10                100
9                 11                100
10                12                 50
11                13                 78
12                14                 66
13                16                 66
14                17                 57
15                18                 71
16                19                 59
17                20                 59
18                21                  0
19                22                100
20                23                 79
21                24                 44
22                25                 89
23                26                  0


In [8]:
 c = cb.CalculateSequenceBreak()

/Users/sgaruda/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [9]:
mathSequenceBreak = c.get_math_seqBreak()
englishSequenceBreak = c.get_eng_seqBreak()
# print(len(mathSequenceBreak[0]))
labels = ['GeneratedPlanID','mathSequenceBreak']
math_df = pd.DataFrame.from_records(mathSequenceBreak,columns = labels)
print(math_df)
labels = ['GeneratedPlanID','englishSequenceBreak']
eng_df = pd.DataFrame.from_records(englishSequenceBreak,columns = labels)
print(eng_df)


     GeneratedPlanID  mathSequenceBreak
0                  1                 71
1                  3                 68
2                  4                100
3                  5                100
4                  6                  0
5                  7                 72
6                  8                 36
7                  9                100
8                 10                100
9                 11                100
10                12                 50
11                13                 78
12                14                 66
13                16                 66
14                17                 57
15                18                 71
16                19                 59
17                20                 59
18                21                  0
19                22                100
20                23                 79
21                24                 44
22                25                 89
23                26                  0


In [10]:
math_merged = pd.merge(vaaData, math_df, on='GeneratedPlanID',how = 'outer')

In [11]:
eng_merged = pd.merge(math_merged, eng_df, on='GeneratedPlanID',how = 'outer')

In [12]:
eng_merged.to_csv("getVAAData_may6.csv")